In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Conv1D, GlobalMaxPooling1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import TensorBoard
from time import time
tensorboard=TensorBoard(log_dir="log/{}".format(time()))

Using TensorFlow backend.


In [2]:
train_data= pd.read_csv("/content/drive/My Drive/Colab Notebooks/Python_LAB2/Lab2_5_Data/train.tsv", sep="\t")
print(train_data.shape)
train_data.head
test_data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Python_LAB2/Lab2_5_Data/test.tsv", sep="\t")
print(train_data.shape)
train_data.head

(156060, 4)
(156060, 4)


<bound method NDFrame.head of         PhraseId  ...  Sentiment
0              1  ...          1
1              2  ...          2
2              3  ...          2
3              4  ...          2
4              5  ...          2
...          ...  ...        ...
156055    156056  ...          2
156056    156057  ...          1
156057    156058  ...          3
156058    156059  ...          2
156059    156060  ...          2

[156060 rows x 4 columns]>

In [0]:
train_data = train_data.drop(columns=['PhraseId', 'SentenceId'])
test_data = test_data.drop(columns=['PhraseId', 'SentenceId'])

In [0]:
label=train_data[['Sentiment']]

In [0]:
train_data=train_data.drop(columns=['Sentiment'])

In [0]:
train_data['Phrase'] = train_data['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))
test_data['Phrase'] = test_data['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x.lower()))

In [0]:
max_fatures =5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train_data['Phrase'].values)
X_train = tokenizer.texts_to_sequences(train_data['Phrase'].values)
X_train = pad_sequences(X_train)

In [0]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(test_data['Phrase'].values)
X_test = tokenizer.texts_to_sequences(test_data['Phrase'].values)
X_test = pad_sequences(X_test)

In [21]:
X_train.shape

(156060, 46)

In [22]:
X_test.shape

(66292, 46)

In [23]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label)
Y_train = to_categorical(integer_encoded)
X_tr, X_te, Y_tr, Y_te = train_test_split(X_train, Y_train, test_size=0.25, random_state=30)
print(X_tr.shape,Y_tr.shape)
print(X_te.shape,Y_te.shape)

(117045, 46) (117045, 5)
(39015, 46) (39015, 5)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
num_classes = Y_train.shape[1]
max_words= X_train.shape[1]
model= Sequential()
model.add(Embedding(max_features,100,input_length=max_words))
model.add(Dropout(0.2))
model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
history=model.fit(X_tr, Y_tr, validation_data=(X_te, Y_te),epochs=5, batch_size=512, verbose=1, callbacks=[tensorboard])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 31s 262us/step - loss: 1.1564 - accuracy: 0.5528 - val_loss: 0.9662 - val_accuracy: 0.6139
Epoch 2/5
117045/117045 [==============================] - 30s 257us/step - loss: 0.9186 - accuracy: 0.6363 - val_loss: 0.9177 - val_accuracy: 0.6310
Epoch 3/5
117045/117045 [==============================] - 30s 260us/step - loss: 0.8569 - accuracy: 0.6598 - val_loss: 0.8810 - val_accuracy: 0.6465
Epoch 4/5
117045/117045 [==============================] - 30s 255us/step - loss: 0.8118 - accuracy: 0.6761 - val_loss: 0.8740 - val_accuracy: 0.6493
Epoch 5/5
117045/117045 [==============================] - 30s 258us/step - loss: 0.7772 - accuracy: 0.6904 - val_loss: 0.8648 - val_accuracy: 0.6539


In [27]:
model.predict_classes(X_te[:1])

array([2])

In [28]:
Y_te[2]

array([0., 0., 1., 0., 0.], dtype=float32)

In [0]:
y_pred=model.predict_classes(X_test[:1])

In [35]:
sub_file = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Python_LAB2/Lab2_5_Data/sampleSubmission.csv',sep=',')
sub_file.iloc[0]

PhraseId     156061
Sentiment         2
Name: 0, dtype: int64

In [40]:
print("PREDICTED LABEL ",y_pred[0])
print("ACTUAL LABEL  ",sub_file['Sentiment'].iloc[0])

PREDICTED LABEL  2
ACTUAL LABEL   2


In [0]:
from keras.optimizers import adam
s=adam(lr=0.001)
model1= Sequential()
model1.add(Embedding(max_features,100,input_length=max_words))
model1.add(Dropout(0.2))
model1.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(128,activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='binary_crossentropy',optimizer=s,metrics=['accuracy'])

In [25]:
history1=model1.fit(X_tr, Y_tr, validation_data=(X_te, Y_te),epochs=5, batch_size=51, verbose=1, callbacks=[tensorboard])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 117045 samples, validate on 39015 samples
Epoch 1/5
117045/117045 [==============================] - 45s 384us/step - loss: 0.3548 - accuracy: 0.8444 - val_loss: 0.3305 - val_accuracy: 0.8520
Epoch 2/5
117045/117045 [==============================] - 44s 377us/step - loss: 0.3237 - accuracy: 0.8564 - val_loss: 0.3247 - val_accuracy: 0.8547
Epoch 3/5
117045/117045 [==============================] - 45s 380us/step - loss: 0.3109 - accuracy: 0.8621 - val_loss: 0.3183 - val_accuracy: 0.8584
Epoch 4/5
117045/117045 [==============================] - 44s 375us/step - loss: 0.3022 - accuracy: 0.8664 - val_loss: 0.3175 - val_accuracy: 0.8582
Epoch 5/5
117045/117045 [==============================] - 44s 377us/step - loss: 0.2952 - accuracy: 0.8699 - val_loss: 0.3179 - val_accuracy: 0.8569
